# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
## 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk import word_tokenize
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

In [2]:
# downloads
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rache\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rache\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rache\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster_response', engine)

# feature
X = df['message']

# targets
Y = df.drop(columns=['id', 'message', 'original', 'genre'])

In [3]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26210    The training demonstrated how to enhance micro...
26211    A suitable candidate has been selected and OCH...
26212    Proshika, operating in Cox's Bazar municipalit...
26213    Some 2,000 women protesting against the conduc...
26214    A radical shift in thinking came about as a re...
Name: message, Length: 26215, dtype: object

## 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    # normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stopwords
    tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens if word not in stopwords.words("english")]
    
    return tokens

In [5]:
X.tolist()[0:10]

['Weather update - a cold front from Cuba that could pass over Haiti',
 'Is the Hurricane over or is it not over',
 'Looking for someone but no name',
 'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
 'says: west side of Haiti, rest of the country today and tonight',
 'Information about the National Palace-',
 'Storm at sacred heart of jesus',
 'Please, we need tents and water. We are in Silo, Thank you!',
 'I would like to receive the messages, thank you',
 'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )']

## 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
# Build pipeline using Random Forest as our classifier first
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ]))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])

## 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)

## 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# Turn preds into a dataframe
Y_pred_df = pd.DataFrame(Y_pred, index=Y_test.index, columns=Y_test.columns)

In [9]:
# Print classification report
print(classification_report(Y_test, Y_pred_df, target_names=Y_test.columns, zero_division=0))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      5001
               request       0.84      0.51      0.63      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.76      0.69      0.72      2700
          medical_help       0.72      0.10      0.17       532
      medical_products       0.79      0.10      0.17       345
     search_and_rescue       0.67      0.07      0.13       165
              security       0.25      0.01      0.02       127
              military       0.81      0.07      0.12       197
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.44      0.58       408
                  food       0.86      0.58      0.69       723
               shelter       0.82      0.35      0.49       590
              clothing       0.77      0.11      0.19        95
                 money       0.82      

## 6. Try improving the model
For example:
* try other machine learning algorithms
* add other features besides the TF-IDF

### Create a function to make testing other models easier

In [10]:
def fit_train_test_model(X, Y, classifier):
    
    # specify the pipeline
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),
        ('clf', MultiOutputClassifier(classifier))
    ])
    
    # Train the pipeline and make predictions
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
    pipeline.fit(X_train, Y_train)
    Y_pred = pipeline.predict(X_test)
    
    # Test our model
    Y_pred_df = pd.DataFrame(Y_pred, index=Y_test.index, columns=Y_test.columns)
    
    print("{} Report:".format(classifier))
    print(classification_report(Y_test, Y_pred_df, target_names=Y_test.columns, zero_division=0))


We can only use algorithms that support multilabal classifications so we'll try out algorithms based on the information on this page: https://scikit-learn.org/stable/modules/multiclass.html

### Decision Tree Classifier

In [8]:
from sklearn.tree import DecisionTreeClassifier

In [16]:
fit_train_test_model(X, Y, DecisionTreeClassifier())

DecisionTreeClassifier() Report:
                        precision    recall  f1-score   support

               related       0.85      0.85      0.85      5001
               request       0.55      0.52      0.54      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.65      0.64      0.64      2700
          medical_help       0.37      0.33      0.35       532
      medical_products       0.42      0.37      0.39       345
     search_and_rescue       0.23      0.21      0.22       165
              security       0.05      0.03      0.04       127
              military       0.32      0.31      0.31       197
           child_alone       0.00      0.00      0.00         0
                 water       0.64      0.66      0.65       408
                  food       0.75      0.72      0.73       723
               shelter       0.61      0.58      0.59       590
              clothing       0.61      0.54      0.57        95
      

### Extra Tree Classifier

In [17]:
from sklearn.tree import ExtraTreeClassifier

In [18]:
fit_train_test_model(X, Y, ExtraTreeClassifier())

ExtraTreeClassifier() Report:
                        precision    recall  f1-score   support

               related       0.84      0.83      0.83      5001
               request       0.50      0.50      0.50      1093
                 offer       0.03      0.03      0.03        32
           aid_related       0.61      0.58      0.59      2700
          medical_help       0.29      0.25      0.27       532
      medical_products       0.34      0.27      0.30       345
     search_and_rescue       0.16      0.15      0.15       165
              security       0.05      0.03      0.04       127
              military       0.32      0.31      0.32       197
           child_alone       0.00      0.00      0.00         0
                 water       0.48      0.40      0.44       408
                  food       0.46      0.43      0.45       723
               shelter       0.42      0.38      0.40       590
              clothing       0.17      0.16      0.16        95
         

### Ensemble Extra Trees Classifier

In [19]:
from sklearn.ensemble import ExtraTreesClassifier

In [20]:
fit_train_test_model(X, Y, ExtraTreesClassifier())

ExtraTreesClassifier() Report:
                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      5001
               request       0.84      0.48      0.61      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.77      0.69      0.72      2700
          medical_help       0.63      0.12      0.20       532
      medical_products       0.70      0.11      0.19       345
     search_and_rescue       0.41      0.04      0.08       165
              security       0.20      0.01      0.02       127
              military       0.56      0.10      0.17       197
           child_alone       0.00      0.00      0.00         0
                 water       0.80      0.30      0.44       408
                  food       0.87      0.45      0.60       723
               shelter       0.80      0.29      0.42       590
              clothing       0.60      0.13      0.21        95
        

### KNeighbors Classifier

In [21]:
from sklearn.neighbors import KNeighborsClassifier

In [22]:
fit_train_test_model(X, Y, KNeighborsClassifier())

KNeighborsClassifier() Report:
                        precision    recall  f1-score   support

               related       0.78      0.98      0.87      5001
               request       0.79      0.07      0.13      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.69      0.04      0.07      2700
          medical_help       0.50      0.00      0.00       532
      medical_products       0.33      0.00      0.01       345
     search_and_rescue       0.00      0.00      0.00       165
              security       0.00      0.00      0.00       127
              military       0.00      0.00      0.00       197
           child_alone       0.00      0.00      0.00         0
                 water       0.71      0.02      0.05       408
                  food       0.75      0.05      0.10       723
               shelter       0.91      0.03      0.07       590
              clothing       1.00      0.04      0.08        95
        

The weighted average F-scores of the models were as follows
* Random Forest: 0.57
* Decision Tree: 0.60
* Extra Tree: 0.51
* Ensemble Extra Trees: 0.56
* KNeighbors: 0.26

## 7. Tune the model
Use grid search to find better parameters. 

In [9]:
# Proceed with our chosen model pipeline
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ]))
    ])),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('features',
   FeatureUnion(transformer_list=[('text_pipeline',
                                   Pipeline(steps=[('vect',
                                                    CountVectorizer(tokenizer=<function tokenize at 0x00000253AC862820>)),
                                                   ('tfidf',
                                                    TfidfTransformer())]))])),
  ('clf', MultiOutputClassifier(estimator=DecisionTreeClassifier()))],
 'verbose': False,
 'features': FeatureUnion(transformer_list=[('text_pipeline',
                                 Pipeline(steps=[('vect',
                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000253AC862820>)),
                                                 ('tfidf',
                                                  TfidfTransformer())]))]),
 'clf': MultiOutputClassifier(estimator=DecisionTreeClassifier()),
 'features__n_jobs': None,
 'features__transformer_

In [11]:
parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__max_depth': [10, 20],
    'clf__estimator__min_samples_split': [2, 5],
    'clf__estimator__ccp_alpha': [0.0, 0.1]
}

In [12]:
cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=3)

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [14]:
cv.fit(X_train, Y_train)

Fitting 2 folds for each of 16 candidates, totalling 32 fits
[CV 1/2] END clf__estimator__ccp_alpha=0.0, clf__estimator__max_depth=10, clf__estimator__min_samples_split=2, features__text_pipeline__vect__ngram_range=(1, 1);, score=0.222 total time= 1.6min
[CV 2/2] END clf__estimator__ccp_alpha=0.0, clf__estimator__max_depth=10, clf__estimator__min_samples_split=2, features__text_pipeline__vect__ngram_range=(1, 1);, score=0.214 total time= 1.6min
[CV 1/2] END clf__estimator__ccp_alpha=0.0, clf__estimator__max_depth=10, clf__estimator__min_samples_split=2, features__text_pipeline__vect__ngram_range=(1, 2);, score=0.213 total time= 1.7min
[CV 2/2] END clf__estimator__ccp_alpha=0.0, clf__estimator__max_depth=10, clf__estimator__min_samples_split=2, features__text_pipeline__vect__ngram_range=(1, 2);, score=0.211 total time= 1.7min
[CV 1/2] END clf__estimator__ccp_alpha=0.0, clf__estimator__max_depth=10, clf__estimator__min_samples_split=5, features__text_pipeline__vect__ngram_range=(1, 1);, 

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x00000253AC862820>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())]))])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=DecisionTreeClassifier()))]),
             param_grid={'clf__estimator__ccp_alpha': [0.0, 0.1],
                         'clf__estimator__max_depth': [10, 20],
                         'clf__estimator__min_samples_split': [2, 5],
                    

In [15]:
cv.best_params_

{'clf__estimator__ccp_alpha': 0.0,
 'clf__estimator__max_depth': 10,
 'clf__estimator__min_samples_split': 2,
 'features__text_pipeline__vect__ngram_range': (1, 1)}

In [16]:
cv.best_score_

0.2181982679201186

## 8. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
Y_pred = cv.predict(X_test)

In [18]:
# Turn preds into a dataframe
Y_pred_df = pd.DataFrame(Y_pred, index=Y_test.index, columns=Y_test.columns)

In [19]:
# Print classification report
print(classification_report(Y_test, Y_pred_df, target_names=Y_test.columns, zero_division=0))

                        precision    recall  f1-score   support

               related       0.79      0.97      0.87      5001
               request       0.77      0.43      0.55      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.69      0.59      0.64      2700
          medical_help       0.54      0.24      0.33       532
      medical_products       0.67      0.32      0.43       345
     search_and_rescue       0.55      0.22      0.31       165
              security       0.14      0.02      0.04       127
              military       0.46      0.31      0.37       197
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.64      0.69       408
                  food       0.80      0.80      0.80       723
               shelter       0.77      0.54      0.64       590
              clothing       0.69      0.48      0.57        95
                 money       0.49      

We've managed to improve the model but only very slightly.

## 9. Export your model as a pickle file

In [20]:
import pickle

In [21]:
with open('final_model.pkl','wb') as f:
    pickle.dump(cv, f)

## 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.